In [14]:
import instructor
from pydantic import BaseModel
from datetime import datetime
from openai import OpenAI
from apscheduler.schedulers.background import BackgroundScheduler
import time
from zoneinfo import ZoneInfo
from tqdm.autonotebook import tqdm

### TakeActionIntructor (Function Call with Intructor)

In [11]:
class TakeActionIntructor:
    def __init__(self) -> None:
        llm = OpenAI(
            api_key="lamacpp", 
            base_url="http://zep.hcmute.fit/7500/v1",  
        )
        self.client = instructor.patch(client=llm)
        self.list_keyword = ["appointment", "remind"]
        self.context = {
            "appointment" : " (Thời gian tính theo ngày giờ Việt Nam, timezone Asia/Ho_Chi_Minh)",
            "remind" : " (Thời gian tính theo ngày giờ Việt Nam, timezone Asia/Ho_Chi_Minh)"
        }
        self.action_class = {
            "appointment" : self.AppointmentDetail,
            "remind" : self.RemindDetail
        }
        
    class AppointmentDetail(BaseModel):
        appointment_datetime: datetime
        appointment_with: str

    class RemindDetail(BaseModel):
        remind_datetime: datetime 
        remind_description: str

    def check_extract_keyword(self, input_string):
        for keyword in self.list_keyword:
            if input_string.lower().startswith(keyword):
                self.keyword = keyword
                return True, input_string[len(self.keyword):].strip()
        return False, input_string
    
    def take_action(self, input_string):
        check, s = self.check_extract_keyword(input_string)
        if not check:
            print("Keyword not found")
            return None
        
        for i in range(5):
            action = self.client.chat.completions.create(
                model="gemma-2-9b-it",
                response_model=self.action_class[self.keyword],
                messages=[
                    {"role": "user", "content": s + self.context[self.keyword]},
                ],
            )
            if ((isinstance(action, self.AppointmentDetail) and action.appointment_datetime.year >= datetime.now().year) 
                or (isinstance(action, self.RemindDetail) and action.remind_datetime.year >= datetime.now().year)):
                return action
        return None

### Test

In [18]:
take_action = TakeActionIntructor()
ta = take_action.take_action("appointment 15h25 14/9/2024 hẹn với bác sĩ Khoa nhé")
if isinstance(ta, take_action.AppointmentDetail):
    print("Appointment: ",ta.appointment_datetime, ta.appointment_with)
elif isinstance(ta, take_action.RemindDetail):
    print("Remind: ", ta.remind_datetime, ta.remind_description)
else:
    print("Year not valid")

Appointment:  2024-09-14 15:25:00+07:00 Bác sĩ Khoa


In [16]:
take_action = TakeActionIntructor()
#take_action.take_action("remind 10h 15/10/2024 chúng ta tiếp tục tư vấn nhé")
count_true = 0
for i in tqdm(range(100)):
    detail = take_action.take_action("remind 10h 17/10/2024 chúng ta tiếp tục tư vấn nhé")
    datetime_true = datetime(2024, 10, 17, 10, 0)
    if detail.remind_datetime.year == datetime_true.year and detail.remind_datetime.month == datetime_true.month and detail.remind_datetime.day == datetime_true.day and detail.remind_datetime.hour == datetime_true.hour and detail.remind_datetime.minute == datetime_true.minute:
        count_true += 1
print(f"Số lần trùng khớp: {count_true}")

100%|██████████| 100/100 [18:23<00:00, 11.03s/it]

Số lần trùng khớp: 100


In [17]:
take_action = TakeActionIntructor()
#take_action.take_action("remind 10h 15/10/2024 chúng ta tiếp tục tư vấn nhé")
count_true = 0
for i in tqdm(range(100)):
    detail = take_action.take_action("appointment Đặt lịch hẹn với bác sĩ Mai Anh Khoa vào lúc 10h ngày 17/09/2024 nhé")
    datetime_true = datetime(2024, 9, 17, 10, 0)
    if detail.appointment_datetime.year == datetime_true.year and detail.appointment_datetime.month == datetime_true.month and detail.appointment_datetime.day == datetime_true.day and detail.appointment_datetime.hour == datetime_true.hour and detail.appointment_datetime.minute == datetime_true.minute:
        count_true += 1
print(f"Số lần trùng khớp: {count_true}")

100%|██████████| 100/100 [22:12<00:00, 13.32s/it]

Số lần trùng khớp: 100


### Example With Schedulers

In [12]:
# Create a scheduler
scheduler = BackgroundScheduler()

# Function to execute
def scheduled_appointment(*args):
    print(f"Bạn có lịch hẹn với {args[0]}")

def scheduled_remind(*args):
    print(f"Nhắc nhở bạn đã tới giờ {args[0]}")

take_action = TakeActionIntructor()
ta = take_action.take_action("remind 23h24 15/09/2024 chúng ta tiếp tục tư vấn nhé")
if isinstance(ta, take_action.AppointmentDetail):
    print("Appointment: ",ta.appointment_datetime, ta.appointment_with)
    ta.appointment_datetime= ta.appointment_datetime.replace(tzinfo=ZoneInfo("Asia/Ho_Chi_Minh"))
    scheduler.add_job(scheduled_appointment, 'date', run_date=ta.appointment_datetime, args=[ta.appointment_with])
elif isinstance(ta, take_action.RemindDetail):
    print("Remind: ", ta.remind_datetime, ta.remind_description)
    ta.remind_datetime= ta.remind_datetime.replace(tzinfo=ZoneInfo("Asia/Ho_Chi_Minh"))
    scheduler.add_job(scheduled_remind, 'date', run_date=ta.remind_datetime, args=[ta.remind_description])

# Start the scheduler
scheduler.start()

# Keep the program running
while True:
    time.sleep(1)


Remind:  2024-09-15 23:24:00+07:00 Tiếp tục tư vấn
Nhắc nhở bạn đã tới giờ Tiếp tục tư vấn


KeyboardInterrupt: 

### Emotion Classification

In [13]:
import instructor
from pydantic import BaseModel
from openai import OpenAI
import enum

class EmotionText(str, enum.Enum):
    """Enumeration for single-label text classification."""
    TIEU_CUC = "Cảm xúc tiêu cực"
    TICH_CUC = "Cảm xúc tích cực"

class Emotion_Classification:
    def __init__(self) -> None:
        llm = OpenAI(
            api_key="lamacpp", 
            base_url="http://zep.hcmute.fit/7500/v1",  
        )
        self.client = instructor.patch(client=llm)
    
    class SingleEmotionPrediction(BaseModel):
        """
        Class for a single emotion class label prediction.
        """

        emotion: EmotionText
        
    def classify(self, data: str):
        return self.client.chat.completions.create(
            model="gemma-2-9b-it",
            response_model=self.SingleEmotionPrediction,
            messages=[
                {
                    "role": "user",
                    "content": f"Cảm xúc tích cực hay tiêu cực qua đoạn văn bản sau: {data}",
                },
            ],
        )

In [14]:
prediction = Emotion_Classification().classify("Tôi đang đau đầu, mệt mỏi, căng thẳng quá, hơi quạo")
print(prediction)

emotion=<EmotionText.TIEU_CUC: 'Cảm xúc tiêu cực'>


### Test

In [15]:
count_true = 0
for i in range(100):
    prediction = Emotion_Classification().classify("Tôi cảm thấy khá tuyệt vọng, muốn buôn xuôi")
    if prediction.emotion == EmotionText.TIEU_CUC:
        count_true += 1
print(count_true)

100


In [10]:
count_true = 0
for i in range(100):
    prediction = Emotion_Classification().classify("Tôi cảm thấy tốt hơn rồi, cảm ơn bạn")
    if prediction.emotion == EmotionText.TICH_CUC:
        count_true += 1
print(count_true)

100
